# **Manipuler une image grâce au code: partie 2**

## Introduction à l'analyse des images - 32M7132

*Adrien Jeanrenaud (adrien.jeanrenaud@unige.ch)*

<div class="alert alert-block alert-info">
<b>Manipulations partie 2</b> : dans la poursuite de nos explorations des images en tant que matrice, nous allons voir comment appliquer des algorithmes, plus ou moins complexes, aux images.
</div>

## **Plan du cours**

> **Corriger les images**
> * L'exposition
> * Le contraste

> **Les filtres et leurs applications**
> * Qu'est-ce qu'un filtre?
> * Attention aux bords
> * Du filtre à l'image
> * Des filtres pour réduire le bruit
> * Des filtres pour détecter les contours

> **La détection sémantique grâce à l'apprentissage profond**
> * Qu'est-ce que l'apprentissage profond?
> * Qu'est-ce qu'un algorithme d'apprentissage profond?
> * Appliquer un algorithme d'apprentissage profond à une image

In [ ]:
# importer les librairies nécessaires

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
# importer une image

image = "images/japanPoster.jpg"
img = cv2.imread(image)

In [ ]:
# Visualiser

color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(color)
plt.show()

In [ ]:
# et en valeurs de gris

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray, cmap="gray")
plt.show()

## **1. Corriger les images**

### 1.1 L'exposition

Il est possible de corriger l'exposition, c'est à dire d'éclaircir ou d'assombrir une image. En utilisant le correction gamma, on contrôle la luminosité en changeant les rations RGB

In [ ]:
from skimage import exposure

?exposure.adjust_gamma

**En dessous de 1, l'image s'éclaircit, en dessus de 1 elle s'assombrit**

In [ ]:
gray_gamma = exposure.adjust_gamma(gray, gamma = 2.25)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_gamma,cmap = "gray")

**La correction fonctionne également sur des images couleurs**

In [ ]:
#image couleur
color_gamma = exposure.adjust_gamma(color, gamma = 2.25)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en couleurs')
ax1.imshow(color)

ax2.set_title ('Image transformée')
ax2.imshow(color_gamma)

<div class="alert alert-block alert-danger">
<b>L'outil est intéressant, enfin si l'on connait notre image et ses besoins.</b>
</div>

### 1.2 Le contraste

Le contraste définit la répartition de lumière dans l'image.
Modifier le contraste de l'image permet d'ouvrir la fenêtre des pixels ; si les valeurs min et max ont peu d'écart, il est possible d'augmenter la rangée des valeurs utilisées

#### 1.2.1 Mofifier le contraste à la main

Modifier le contraste c'est appliquer une transformation point par point. On peut le faire à la main en changeant chaque point.

#### Modifier le contraste des images en valeurs de gris à la main

In [ ]:
# Trouver les valeurs min et max 

ma = gray.max()
mi = gray.min()
print(mi,ma)

In [ ]:
# Convertir l'image en float et ouvrir la fenêtre de valeurs

c = gray.astype(float)
gray_c = 255.0*(c-mi)/(ma-mi+0.0000001).astype(int)

In [ ]:
# Est-ce que ça a bien fonctionné ?

ma1 = gray_c.max()
mi1 = gray_c.min()
print(mi1,ma1)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_c,cmap = "gray")

#### Modifier le contraste des images en couleurs

In [ ]:
# On divise l'image en différents r, g, b 

r,g,b = cv2.split(img)

In [ ]:
ma = r.max()
mi = r.min()
print(mi,ma)
c = r.astype(float)
im1r = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
ma = g.max()
mi = g.min()
print(mi,ma)
c = g.astype(float)
im1g = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
ma = b.max()
mi = b.min()
print(mi,ma)
c = b.astype(float)
im1b = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
# On remet les canaux ensemble
# Attention à l'ordre des canaux de couleurs

color_c = cv2.merge([im1b, im1g, im1r]).astype(int)


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image couleur')
ax1.imshow(color)

ax2.set_title ('Image transformée')
ax2.imshow(color_c)

<div class="alert alert-block alert-danger">
<b>Ces deux opérations sont là pour montrer qu'il est possible de modifier les valeurs de la matrice point par point.</b>
</div>

## **2. Les filtres et leurs applications**

### 2.1 Qu'est-ce qu'un filtre? ?

Dans le traitement numérique des images, les filtres sont utilisés principalement pour flouter, améliorer la netteté ou détecter les contours d'une image. Le filtre permet de supprimer les impurtées, le plus souvent il prépare l'image en vue d'opérations plus poussées, comme pour l'apprentissage profond. 

Un filtre est une petite matrice de dimension impair (3x3, 9x9, etc.) qui s'applique par convolution à l'image. Une convolution est simplement un opération matriciel entre le filtre et l'image. Le filtre (une matrice de 3x3 par exemple), se déplace sur l'image et une nouvelle image est obtenue lorsque l'opération est effectuée sur chaque pixel.
L'opération de convolution réduit la taille de l'image, à moins que des bords soient ajoutés. 

<img src="https://miro.medium.com/proxy/0*YfpMfPnz6n2g4vIz.jpg" title="filtre"/>

<img src="https://www.researchgate.net/profile/William-Overell/publication/315478232/figure/fig30/AS:671530967113758@1537116867792/Concept-of-convolution-Our-kernel-mask-is-shown-below-as-M-From.ppm" title="filtre2"/>

### 2.2 Attention aux bords

Le déplacement du filtre sur l'image pose la question du traitement des bords, car les pixels aux extrémités de l'image doivent avoir le même traitement que les autres ; c'est-à-dire qu'ils doivent passer par toutes les composantes du filtre. Dans ce cas, il y a plusieurs méthodes pour élargir les bords de l'image afin que tous les pixels de l'image de base soient traités correctement. 

Le processus de création de données en dehors de l'image s'appelle en anglais "padding".

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F3232%2F1*9reDuDh3nXs_kJ-M4eq0Ow.png&f=1&nofb=1&ipt=5e8431d61861a6afe5ece799945d9a7a3841a1e80ad8eb42c80e8c5b4f567755&ipo=images" title="padding"/>


**Principalement, il y a**
> * ajout de zéros
> * constante arbitraire
> * plus proche voisin
> * en miroir 
> * reprend le bors opposé

### 2.3 Du filtre à l'image

Il existe plusieurs types de filtres dont le paramétrage et les effets sont bien connus. Avant de les aborder, regardons comment l'on peut simplement créer et appliquer un fitlre sur notre image.

https://setosa.io/ev/image-kernels/

In [ ]:
# La fonction convolve de la librairie Scipy nous permet d'opérer une convolution sur une image 
# Regardons ses paramètres

import scipy.ndimage

?scipy.ndimage.filters.convolve

**Il nous faut principalement une image (input), un filtre (weights) et un choix de padding (mode)**


**À vous de faire**

In [ ]:
# Créer un filtre, 3x3 par exemple

filtre = np.array([[1,0,1],
                  [0,1,0],
                  [1,0,1]])
print(filtre.shape, "\n", filtre)

In [ ]:
# Paramétrer la convolution

grayf = gray.copy()

gray_filtre = scipy.ndimage.filters.convolve(grayf, filtre, mode="reflect")

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_filtre,cmap = "gray")

Nous avons créé un filtre au hasard. Il existe des filtres bien connus.

<img src="https://www.researchgate.net/profile/Olivia-Cheung-2/publication/235368613/figure/fig1/AS:272536512495617@1441989176053/An-image-filtered-to-include-A-both-low-and-high-spatial-frequency-information-B.png" title="freq1"/>

### 2.4 Des filtres pour réduire le bruit

Egaliser un histogramme permet d'affiner le contraste d'une image en distribuant mieux les valeurs, de cette manière l'intensité est mieux répartie. Il existe plusieurs filtres bien connus dont:

#### Filtre médian

Le filtre médian permet de réduire le bruit tout en conservant les contours, il est souvent utilisé pour supprimer le bruit sel et poivre (incursion de pixels noirs et blancs dans l'image) ; chaque pixel est remplacé par la médiane de son voisinage et cela permet de supprimer les valeurs abberantes. Le filtre médian garde le contraste, la luminosité et les contours

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.stack.imgur.com%2Fdw60I.png&f=1&nofb=1&ipt=98a04cc296528ada2699bafee8ef3eafe1af5ab35f7a2d6743a7659fdbe48dc8&ipo=images" title="median"/>

In [ ]:
?scipy.ndimage.filters.median_filter

In [ ]:
# Paramétrage de la fonction
# La fonction prend principalement en paramètres une image (input), la taille du filtre (size) et le type de padding (mode)

gray_mm = gray.copy()

gray_median = scipy.ndimage.filters.median_filter(gray_mm, size=9)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_median,cmap = "gray")

#### Filtre Gaussien

Le filtre gaussien, comme son nom l'indique suit une distribution gaussienne, c'est-à-dire une loi normale centrée et réduite. Le sigma définit la forme de la cloche, et dans le traitement de l'image cela signifie que le bruit peut être réduit (sigma < 1) ou que le flou peut être accentué (sigma > 1).

Le filtre gaussien vient lisser les imperfection de l'image, les détails et les contours sont atténués.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Generalized_normal_densities.svg/langfr-560px-Generalized_normal_densities.svg.png" title="gauss"/>

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.JpX8ONYKxZmIBAHIi1KpjAHaIy%26pid%3DApi&f=1&ipt=f751c30b941d608cdf6715c0829c2abc7c2d6f08aa83e0155fe4bef8ebc48ea9&ipo=images" title="gauss2"/>

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fsupport.cognex.com%2Fdocs%2Fcvl_900%2Fweb%2FEN%2Fcvl_vision_tools%2FContent%2FImages%2F18_8.jpg&f=1&nofb=1&ipt=0ba811ae846f092e40b20e9c7f1d74fc7f1e0f88240a48ec1ce7483ca4f1017b&ipo=images" title="gauss3"/>

In [ ]:
?scipy.ndimage.filters.gaussian_filter

In [ ]:
# Paramétrage de la fonction
# La fonction prend principalement une image (input) et un sigma

gray_g = gray.copy()

gray_gauss = scipy.ndimage.filters.gaussian_filter(gray_g, sigma=10)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_gauss,cmap = "gray")

In [ ]:
# Il existe également une fonction similaire dans la librairie OpenCV

?cv2.GaussianBlur

In [ ]:
# Paramétrage de la fonction
# La fonction prend principalement une image, une taille de filtre (ksize) et un sigma

gray_gcv = gray.copy()

gray_gausscv = cv2.GaussianBlur(gray_gcv, (19,19), 10)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_gausscv,cmap = "gray")

**Pour aller plus loin: https://setosa.io/ev/image-kernels/**

<div class="alert alert-block alert-warning">
<b>Exercice</b>: à partir de l'image et de la détection des visages ci-dessous, il vous faudra flouter ces visages et enregistrer l'image.</div>

In [ ]:
# télécharger une image

import requests 

DownURL = "https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fwww.bluebird-electric.net%2Facademia%2Facademia_pictures%2FUniversity_of_Geneva_Universite_de_Geneve_Switzerland_Planet_Solar.jpg&f=1&nofb=1&ipt=47cb55e9bf396b35de8f272546914dbe02979fdd2163fd7e2e15601bec35314c&ipo=images" # choix de l'URL
img_data = requests.get(DownURL).content # télécharger
with open('unige.jpg', 'wb') as handler: # définir le fichier et son chemin
    handler.write(img_data) # enregistrer l'image

In [ ]:
# import notre image

path = "unige.jpg"
image = cv2.imread(path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
# Appliquer un algorithme simple
# https://towardsdatascience.com/viola-jones-algorithm-and-haar-cascade-classifier-ee3bfb19f7d8

grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml") #https://github.com/opencv/opencv/tree/master/data/haarcascades
detected_faces = face_cascade.detectMultiScale(grayscale_image)

In [ ]:
# Dessiner les lignes

for (column, row, width, height) in detected_faces:
    image = cv2.rectangle(image,(column, row),(column + width, row + height),(0, 255, 0),4)
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
# flouter les visages

### 2.5 Des filtres pour détecter les contours

Dans une image, un contour se comprend comme la différence d'intensité entre deux pixels. La détection de contours cherche le changement soudain entre deux valeurs de pixel.

Mathématiquement le calcul d'intensité se fait par l'utilisation de dérivées : en somme, l'application de une ou deux dérivées permet d'accentuer le contraste entre deux parties de l'image sensées décrire un contour. 

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi2.wp.com%2Fwww.adeveloperdiary.com%2Fwp-content%2Fuploads%2F2019%2F05%2FHow-to-implement-Sobel-edge-detection-using-Python-from-scratch-adeveloperdiary.com-sobel.jpg%3Fresize%3D600%252C281&f=1&nofb=1&ipt=429d15d804fa52c46efde3a1ffa086240b1a58a4d32969c28559b32185b03d89&ipo=images" title="edge"/>

Il existe plusieurs types de filtres bien connus pour la détection des contours, nous allons en voir certains:

* Prewitt
* Sobel
* Canny
* Laplacien
* Laplacien Gaussien


Sans trop rentrer dans les détails, la détection de contours se divise principalement en deux méthodes. D'un côté une méthode basée sur le gradient via la première dérivée : le gradient est le taux de varaiation d'un point en fonction de ses voisins. D'un autre côté, une méthode basée sur une gaussienne à l'aide d'une derivée de second ordre. 

#### Filtre Canny

Le filtre Canny permet de détecter les contours (sans division horizontale et verticale préalable, comme le Sobel et Prewitt). Il est performant dans la détection (contours faibles et forts) et dans la localisation des contours (faible erreur entre contours détectés et contours réels). Malgré sa performance, son coup d'utilisation est non néglibeable.

En d'autres termes, voilà le processus simplifié du filtre Canny:

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fimage.slidesharecdn.com%2Fe2822eef-6993-4540-9321-65ca5f35eb39-161009120200%2F95%2Fexploring-methods-to-improve-edge-detection-with-canny-algorithm-13-638.jpg%3Fcb%3D1476014597&f=1&nofb=1&ipt=f300f199eaf34361e890d50c9054e9b38c9b25b29d6863942c7ec716bbdbc49b&ipo=images" title="k-means2"/>


In [ ]:
?cv2.Canny

**La fonction prend principalement comme arguments: une image, un seuil bas et un seuil haut**

> * En dessous du seuil bas, le pixel est rejeté
> * En dessus du seuil haut, le pixel est considéré comme un contour
> * Entre, si le pixel est connecté au seuil haut, il est accepté

In [ ]:
# Paramétrage de la fonction

gray_c = gray.copy()

gray_canny = cv2.Canny(gray_c, 100, 200)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_canny,cmap = "gray")

#### Laplacien Gaussien

Le filtre laplacien gaussien: la gaussienne est ajoutée au filtre laplacien afin de reduire le bruit dû à l'application du filtre par l'image. La méthode fonctionne particulièrement bien lorsque la transititon est abrupte. 

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fimage3.slideserve.com%2F5732879%2Flaplacian-of-gaussian-log-l.jpg&f=1&nofb=1&ipt=7de63d7d515aa0b0f2deab78c16be380dedb1cc5b2b9077ca489ce5c8171cfc5&ipo=images" title="lg"/>

In [ ]:
?scipy.ndimage.filters.gaussian_laplace

**La fonction prend principalement comme arguments: une image, un sigma et un padding**

> * Le mode de padding, comme vu précédemment
> * Le sigma fonctionne un peu comme un seuil, plus il est élevé plus on perd de points saillants

In [ ]:
# Paramétrage de la fonction

gray_lg = gray.copy()

gray_laplace_gauss = scipy.ndimage.filters.gaussian_laplace(gray_lg, sigma=2, mode="reflect")

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_laplace_gauss,cmap = "gray")

## 3. La détection sémantique grâce à l'apprentissage profond


### 3.1 Qu'est-ce que l'apprentissage profond?

L'apprentissage profond pour les images consiste à former des réseaux de neurones artificiels pour comprendre, reconnaître et extraire des informations à partir d'images. Ces réseaux apprennent à détecter des motifs visuels complexes, permettant par exemple la reconnaissance d'objets, la segmentation d'éléments dans une image ou encore la génération de contenu visuel.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi0.wp.com%2Fdevelopersbreach.com%2Fwp-content%2Fuploads%2F2020%2F08%2Fcnn_banner.png%3Ffit%3D1200%252C564%26ssl%3D1&f=1&nofb=1&ipt=210f2ef4bf2dc42949511533ba31820f74019d5bf5e49e6fc99d5dd612d5877e&ipo=images" title="cnn"/>


Dans les réseaux de neurones convolutionnels, les filtres aident à détecter des motifs dans les images. Ils agissent comme des détecteurs de caractéristiques, captant des éléments tels que des bords, textures ou formes. En se superposant à l'image, ces filtres permettent au réseau de neurones d'extraire des informations clés pour reconnaître et interpréter son contenu.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.mdpi.com%2Finformation%2Finformation-10-00375%2Farticle_deploy%2Fhtml%2Fimages%2Finformation-10-00375-g002.png&f=1&nofb=1&ipt=2131ac2d9adeec1a01b6e21dd5aca4b2ffd370609b7a1f4bd8dbf238c7c5734b&ipo=images" title="conv"/>

### 3.2 Qu'est-ce qu'un algorithme d'apprentissage profond?

Il existe plusieurs algorithmes d'apprentissage profond utilisés en traitement d'images. Parmi les plus populaires :

> * Réseaux de neurones convolutionnels (CNN) : Les CNN utilisent des filtres pour extraire progressivement des caractéristiques des images, en passant de traits simples (comme les bords) à des attributs complexes pour la classification ou la détection d'objets.
Ils sont spécifiquement conçus pour traiter des données structurées en grille comme les images. Les CNN sont largement utilisés pour la classification d'images, la détection d'objets, la segmentation sémantique et la génération d'images.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse4.mm.bing.net%2Fth%3Fid%3DOIP.F5A9npwC1qzLEn-rV59WlAHaDq%26pid%3DApi&f=1&ipt=b681764f327db8665c1f7bc338f6f0ae270e6d84572f2ba13082962b4d320968&ipo=images" title="cnn2"/>

> * Réseaux antagonistes génératifs (GAN) : Les GAN consistent en deux réseaux adverses, un générateur créant des images et un discriminateur les évaluant ; ils s'améliorent mutuellement pour produire des images réalistes.
Ils sont utilisés pour créer des images synthétiques réalistes et pour l'augmentation de données.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F8000%2F1*et3fMPDclTv6ZQSf1xbkag.jpeg&f=1&nofb=1&ipt=7e8b028485afc19c8094e8b345278b0c2ade5ea51ea58433a0f1f8cedd8c436d&ipo=images" title="gan"/>

> * Réseaux neuronaux récurrents (RNN) : Bien que plus utilisés pour le traitement du langage naturel, les RNN utilisent des connexions cycliques pour traiter des données séquentielles comme des vidéos, prédisant ou générant des images basées sur des séquences précédentes.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F1400%2F1*5bjD7kmtaJI-n3qztBC2Ig.png&f=1&nofb=1&ipt=4abd920e335f1c5f5da5ec46903edd21f74d2c661fdb1c9dc3566ad84c434804&ipo=images" title="rnn"/>


#### En termes d'applications en traitement d'images :

> * Classification d'images : Les CNN sont utilisés pour identifier et classer des objets dans des images, comme dans les applications de reconnaissance faciale, la détection de maladies à partir d'images médicales, etc.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F4096%2F1*nR5QCdmqUnvU2JFBu2Xa-Q.png&f=1&nofb=1&ipt=ea3c1fbb7dc192ac04b0a1fe2c485896118571d8ca24ba5b5c75b1e332fc3258&ipo=images" title="ci"/>

> * Détection d'objets : Les CNN, notamment avec des architectures comme YOLO (You Only Look Once) ou SSD (Single Shot MultiBox Detector), sont employés pour localiser et identifier plusieurs objets dans une seule image.

<img src="https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fpjreddie.com%2Fmedia%2Fimage%2FScreen_Shot_2016-11-17_at_11.14.54_AM.png&f=1&nofb=1&ipt=bf73934fbefc8bb335cdbb110a4e97e3f91e6e7ea1e31908b88c6d45c4a9b7f0&ipo=images" title="do"/>

> * Segmentation sémantique : Les réseaux tels que U-Net sont utilisés pour segmenter chaque pixel d'une image en catégories spécifiques, comme la délimitation des organes dans des images médicales.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.redd.it%2Fapse69opd3a61.jpg&f=1&nofb=1&ipt=4af80a8b267ff98907ed31358c760f7de9a04027d1658ccd6939241275e62f96&ipo=images" title="rnn"/>

> * Génération d'images : Les GAN sont utilisés pour générer des images réalistes, comme la génération de visages, de paysages ou même de créations artistiques.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fventurebeat.com%2Fwp-content%2Fuploads%2F2019%2F12%2F1_BU2GnLJF1AcrkhvbCHdppw-e1577390193648.jpeg%3Fw%3D1200%26strip%3Dall&f=1&nofb=1&ipt=67e8f68acbf4764ea54315b9863530bd32bac48a175fe184f8c9ab91aebc262f&ipo=images" title="rnn"/>


Ces algorithmes sont appliqués dans de nombreux domaines, de la médecine à la vision par ordinateur en passant par la surveillance et la robotique.

### 3.3 Appliquer un algorithme d'apprentissage profond à une image

Pour des raisons techniques nous allons nous déplacer sur Colab:

https://colab.research.google.com/drive/11N8wwu2B9eXrV6gs8wNRq7X761CKEhvE?usp=share_link
